<a href="https://colab.research.google.com/github/MorningMoment/Python_Material/blob/master/Day_2_data_(datetime).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computing Bootcamp II -- Data Processing (numpy & pandas)


Ziwei Wang - zwwang@uchicago.edu

In [1]:
### Load python packages
import numpy as np 
import pandas as pd
import os
import fnmatch
import h5py                             # hdf5 or .nc4 (this is 200x faster than netCDF4 for .nc4, does not work for .nc files)
import imageio                          # load png into numpy array
from skimage import io                  # for tif files
from sklearn import datasets as skdata  # Toy Datasets
import multiprocessing                  # Simple multi-processing
import matplotlib.pyplot as plt         

# To install netCDF4 for .nc files (does not come pre-loaded in CO)
!pip install netCDF4
from netCDF4 import Dataset

### To mount google drive in runtime to access files in the drive so you can access files
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 4.1MB 2.8MB/s 
     |████████████████████████████████| 307kB 46.1MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
### Change the current working directory to folder on drive
homedir = '/content/drive/My Drive/my_bootcamp/'   ## if you named your local folder 'my_bootcamp'
sharedir = '/content/drive/My Drive/cfr_bootcamp/'  ## folder shared through Google Drive link on Github page
os.chdir(homedir)
print("current directory is : " + homedir)

loaddir = '/content/drive/My Drive/cfr_bootcamp/Data/'      
print("load directory is : " + loaddir)

current directory is : /content/drive/My Drive/my_bootcamp/
load directory is : /content/drive/My Drive/cfr_bootcamp/Data/


## Navigating Google Colaboratory

Try out the new code editor! You will be prompted to use that every time you open a Colab notebook. 
*   Auto completion of codes. 
*   Documentation of functions. 

### Most shell commands work in the following fashion 
*   %cd *dir*
*   !ls
*   !pwd

In [0]:
# to view dataframes interactivly
%load_ext google.colab.data_table

### Magic commands
Cell magic commands from IPython come with Colab Notebook. <br>
https://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb

In [0]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%perl 

A simple example of what you could do with cell magic (%%time). We will cover cython magic later (Time-saving in numpy). 

In [0]:
# Initialize a random matrix with specified dimensions
a = np.random.random_sample((30, 20))


In [0]:
# 'Magic' command in ipython notebook cells
%%time
sz = np.shape(a)
print(sz)
sum = 0.
for row in range(sz[0]):
  for col in range(sz[1]):
    sum = sum+a[row,col]


(30, 20)
CPU times: user 1.05 ms, sys: 10 µs, total: 1.06 ms
Wall time: 1.48 ms


Wall time is the actual time, usually measured in seconds, that a program takes to run or to execute its assigned tasks. <br>
*Reference*:
https://whatis.techtarget.com/definition/wall-time-real-world-time-or-wall-clock-time

In [0]:
%%html
<marquee style='width: 80%; color: blue;'><b>Whee!</b></marquee>

## Introduction: Showcase of Research Data

**Step1** (Not long-form data): Data prepared for calculation step. 

In [0]:
# Loading numpy file from Python2 under Python3 environment
step1_data = np.load(loaddir+'climate_data/USM00070026_2015_cape.npy',allow_pickle=True,encoding='latin1').tolist()
#print(step1_data.keys())
df_era_cape = pd.DataFrame.from_records(step1_data['era'])
#df_era_cape.head()

**Step2** (Long-form data): Data prepared for analysis. Calculation can compress the irrelevant data to some simple values. 

In [0]:
df = pd.read_csv(loaddir+'climate_data/all_cape_long_form.csv',index_col=0)
#df.head(20)



---



---



---



---



# Day 1: Numpy & Gridded Data

Goals of this section: 

1.   Introducing numpy: Initializing array and ndarray. 
2.   Processing data from different sources (csv, npy, netcdf, HDF5).
3.   Numpy built-in functions. 
3.   End-of-class practice: Calculate mean and anomaly along certain dimensions (time/spatial). 


A good example to follow for introduction to numpy: <br> 
https://notebooks.azure.com/wesm/projects/python-for-data-analysis/html/appa.ipynb





**Tip: Initalization of arrays is always the slowest step. Minimize how many times you are creating arrays.** --> manipulate and write-to arrays whenever possible

numpy arrays are '0' indexed (the first item, dimension, etc. is the zeroth item)





## Initializing array

1. Make a numpy array full of zeros/ones. 
print(np.zeros((10)))
print(np.ones((10)))

2. Make a ndarray full of zeros/ones/np.nan.
a = np.zeros((3,3))
a.fill(np.nan)

3. Initialize an array of integers placed in ascending order. 

In [0]:
# Create Arrays
a = np.ones((5,10),dtype=int)
a.dtype


dtype('int64')

In [0]:
# np.linspace
np.linspace(0,9,10)
#Use dtype to change the output format, default is float
print(np.linspace(0,9,10,dtype=int))



[0 1 2 3 4 5 6 7 8 9]


In [0]:

# np.arange
# arange starts at zero by default
print(np.arange(5))
print(np.arange(0,10,dtype=float))

[0 1 2 3 4]
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [0]:
#Construct a array
print(np.zeros((10)))
print(np.ones((10)))

#Construct a ndarray
print('Construct 3x3 ndarray')
print(np.zeros((3,3)))

#Equivalent to
a = np.zeros(10)
print('Before Filling with any specified element: ')
print(a)
a.fill(np.nan)
print('Before Filling with 1: ')
print(a)

#Initialize array with np.nan values
a.fill(np.nan)
print('Initialize array with np.nan: ')
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Construct 3x3 ndarray
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Before Filling with any specified element: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Before Filling with 1: 
[nan nan nan nan nan nan nan nan nan nan]
Initialize array with np.nan: 
[nan nan nan nan nan nan nan nan nan nan]


In [0]:
#Initialize array with np.nan values
a = np.zeros(10)
a.fill(np.nan)
print('Initialize array with np.nan: ')
print(a)


Initialize array with np.nan: 
[nan nan nan nan nan nan nan nan nan nan]


In [0]:
# Initializing a random matrix with specified dimensions
# np.random.randn: Return a sample (or samples) from the “standard normal” distribution.
a = np.random.randn(5,2)
a

first, second, thrid = np.split(a, [1,3])
print('This is the first part')
print(first)
print('This is the second part')
print(second)
print('This is the third part')
print(third)

This is the first part
[[1.52267581 1.22357696]]
This is the second part
[[ 0.02198187 -1.31810598]
 [-0.70440922 -0.06171276]]
This is the third part


NameError: ignored

Check data type; size and shape of np arrays. 


1.   type()
2.   np.shape()
3.   np.size()/len()



In [0]:
### Inspect arrays ###
a = np.random.randn(5,2)
type(a)


In [0]:
### Find the dimensions of your array ###
a.shape

In [0]:
### For a list object ###
l = [0,1,2,3]
ll = np.array(l)
ll

Converting a list from and to numpy.


In [0]:
# Convert the ndarray back to list
del l
l = list(ll)
l

## Slicing numpy array


Use colon to slice array.

*   Keep all elements (Use colon along).
*   Explicitly specify start and end element.
*   Specify the step/increment. 






In [0]:
# Slicing arrays
arr = np.arange((12))
a = np.arange((12)).reshape((6,2))
a[1:5,:]
#print(arr[0:8:2])


In [0]:

# Specifying starting element and increment
# !!! Explain the double colon !!!
a[[1,3,4],:]
loc = np.where(a > 4)
a[loc]
# Slice the last elements 


Reverse the array: 

*   Use np.flip
*   Use ::-1



In [0]:
#Reverse the array
print(arr[::-1])
print(np.flip(arr))



Slice the array using customized conditions

In [0]:
#Pick out the ones that are odd numbers
loc = (arr % 2 == 1)
print(arr[(arr % 2 == 0)])
#Pick out the ones larger than certain number
print(arr[np.where(arr > 3)])

## Reading in Data: numpy, csv, netcdf (hdf5), png, tiff

Reading and saving in npy form.

In [0]:
### For npy ###
# Save data to npy form
arr = np.arange(12).reshape((4,3))
np.save(os.path.join(loaddir,'test_save.npy'),arr)
#p.save(homedir+'test_save.npy',arr)

In [0]:
# Try to load the data
del arr
arr = np.load(homedir+'test_save.npy')
arr

In [0]:
### For HDF5 ###
filename1 = 'Monthly_temperature_climate_model_CCSM4.nc4'
data = h5py.File(os.path.join(loaddir+'climate_data/',filename1),'r')
# Show keys of loaded file
#print(list(data.keys()))

### For netcdf ###
# Please use netCDF4 (Dataset) if you want to use reanalysis 
f = Dataset(loaddir+'reanalysis_data/era5_global_201507.nc','r')
# Print all keywords
#print(f)
#lon = f.variables['longitude']
#lat = f.variables['latitude']

### For jpg & png ###
face = imageio.imread(loaddir+'climate_data/temperture_color.png')
plt.imshow(face)
plt.show()
plt.axis('off')

### For tiff ###
img = io.imread(loaddir+'crop_data/maize_tiff/maize_DataQuality_HarvestedArea.tif')
#img = io.imread(loaddir+'NDVI.tif')
plt.imshow(img, cmap='gray')
plt.axis('off')

Read example csv file from sample data (California Housing datset). 

In [0]:
# Read csv files
filename = '/content/sample_data/california_housing_test.csv'
data = pd.read_csv(filename,sep=',') #nrows, index_col (Discuss later), 
data

Getting data from url (Github here):

In [0]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)
state_data


## Exercise: numpy operation 
1. load in 'era5_land_201501_2x2.nc', located under loaddir+'reanalysis_data/'
2. print last 10 items (slice the array)
3. print the mean values of selected variable: Time mean (array of 2D map); Spatial mean (array of time series)



In [0]:
## blank cell
data  = Dataset(loaddir+'reanalysis_data/era5_land_201501_2x2.nc','r')
t2m = data.variables['t2m']
print(np.shape(t2m))
print(t2m[-1,:,:])


## Concatenating Arrays

In [0]:
arr1 = [0,1,2]
arr2 = [3,4,5]
# np.hstack
print('Stack arrays in sequence horizontally (column wise).')
#np.hstack([arr1,arr2])

# np.vstack and np.stack(..., axis=0)
print('Stack arrays in sequence vertically (row wise).')
np.stack([arr1,arr2],axis=1)

### Looping through a directory

In [0]:
rootdir = loaddir+'crop_data/yield_simulations/'
filelist  = os.listdir(rootdir)
files     = fnmatch.filter(filelist, '*W0*.nc4')

Y = np.zeros((len(files),360,720)) # most geospatial data is [time, lat, lon] or [time, y, x]
for i,myfile in enumerate(files): 
for myfile in files:
    filepath = '%s/%s'%(rootdir,myfile)
    y = h5py.File(filepath,'r')['yield_mai']
    Y[i,:,:] = np.ma.mean(y[1:30,:,:], axis=0)

np.where(Y[0,:,:] < 1000)

## Time saving methods (multi-processing and cython)

One of the slowest operations of any data analysis is opening files. 

In [0]:
def useless(T):
    rootdir = '/content/drive/My Drive/cfr_bootcamp/Data/crop_data/yield_simulations'
    f = h5py.File(rootdir+'/pdssat_agmerra_fullharm_yield_mai_global_annual_1980_2010_C360_T'+T+'_W0_N200_A0.nc4','r')
    a = f['yield_mai'][:]#.filled(0)
    a_sum = 0
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for n in range(a.shape[2]):
                a_sum = a_sum+a[i,j,n]
    #a_cum.tofile(T+'.bin')
    print(T)
    return a_sum

In [0]:
%%time
T = ['-1','0','1','2','3','4','6']
for tt in T: y = useless(tt)
  

Install and load cython package first. 

In [0]:
#!pip install cython
%load_ext Cython

Use cython with data type of variables specifically defined. 

In [0]:
%%cython
import h5py

def useless_cython(T):
    # define types of variables for optimization
    cdef int i, j, n
    cdef double a_cum
    ############################################

    rootdir = '/content/drive/My Drive/cfr_bootcamp/Data/crop_data/yield_simulations'
    f = h5py.File(rootdir+'/pdssat_agmerra_fullharm_yield_mai_global_annual_1980_2010_C360_T'+T+'_W0_N200_A0.nc4','r')
    a = f['yield_mai'][:]
    a_sum = 0
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for n in range(a.shape[2]):
                a_sum = a_sum+a[i,j,n]
    #a_cum.tofile(T+'.bin')
    print(T)
    return a_sum

In [0]:
%%time
T = ['-1','0','1','2','3','4','6']
for tt in T: y = useless_cython(tt)

One snapshot of time comsumption.

*   Python: 18.2s.
*   Multiprocessing: 19.6s. (Removed now)
*   Cython: 17.2s.




**Conclusion:** For the example shown above, using cython saves about 10% of time it takes to load and finish specified calculation of several nc files.  <br>
I have tested on a larger research project with cython vs python, it can improve computational efficiency by 20% for a wrf-python package function.


## Time saving in numpy (avoiding for-loops)

### 1. Use the built-in numpy functions
All the built-in functions in numpy (too many to count) are each essentially to save a for loop... and are almost always the fastest way to do your operation.

1. sum/standard deviation/mean
2. difference
3. quantiles
4. sort

Example in this section make code run faster (any tricks possible)

In [0]:
# Print the array we initialized before
arr = np.arange(12).reshape((4,3))
arr
# Start exploring numpy built-in functions
# 1. np.sum

# 2. np.std

# axis = 0; axis = 1



In [0]:
arr = np.asarray(arr,dtype=float)
# mask out some elements to np.nan
arr[3,:] = np.nan
# 3. np.mean/np.nanmean 



In [0]:
# np.difference 
np.diff(arr,axis=0) # axis=0: difference row-wise; axis-1: difference col-wise

For np.quantile/np.percentile, initialize a random array with integers. 

In [0]:
# Use randint for percentile
arr = np.random.randint(10,size=11)

In [0]:
print(sorted(arr))
np.percentile(arr,20)
# Equivalent to np.quantile (0~1)


Sorting the array directly and indirectly. 

In [0]:
arr = np.random.randint(10,size=12).reshape((4, 3))
print('Original array: ')
print(arr)


In [0]:
# This will happen inplace
arr.sort(axis=0)
print('Array after sorting: ')
print(arr)

In [0]:
# This will not affect order in arr
arr1 = np.sort(arr,axis=1)
arr1

In [0]:
### Sorting the array indirectly
arr = np.array([5, 0, 1, 3, 2])
# Sort by the index
indexer = arr.argsort() #default is ascending
print('The index of sorted array: ')
print(indexer)
print('Array sorted indirectly: ')
print(arr[indexer])


Recall the example we did to compare python, multiprocessing and cython. Now let's replace for-loops with np.sum. 

In [0]:
# Now replace for-loop with np.sum
def useful(T):
    rootdir = '/content/drive/My Drive/cfr_bootcamp/Data/crop_data/yield_simulations'
    f = h5py.File(rootdir+'/pdssat_agmerra_fullharm_yield_mai_global_annual_1980_2010_C360_T'+T+'_W0_N200_A0.nc4','r')
    a = f['yield_mai'][:]#.filled(0)
    a_sum = 0 #This line is kept to keep consistency with previous function
    a_sum = np.sum(a)
    print(T)
    return a_sum

In [0]:
%%time
T = ['-1','0','1','2','3','4','6']
for tt in T: y = useful(tt)

**Conclusion**: Using numpy built-in functions could save you a lot of time. 

### 2. Use np.tile or np.repeat

In [0]:
arr = np.arange((12)).reshape((4,3))
print(arr)

row_mean = np.mean(arr,axis=1)
print(row_mean)

Here are examples of ndarray computation: The dimensions have to match. 

In [0]:
# To add a newaxis at specified position
arr_3d = arr[:, np.newaxis, :]
arr_3d.shape


In [0]:
# The following line will produce an error
#print(arr - row_mean)
# Instead, use np.newaxis when the broadcasting confuses the program.
print(arr - row_mean[:,np.newaxis])

#The following is one standard way with np.repeat (Above line is suggested)
print('Anomaly along the second dimension: ')
anom_matrix = arr - np.repeat(row_mean[:,np.newaxis],3,axis=1) # repeat for 3 times, axis = 1
print(anom_matrix)


See example from numpy.tile: 
https://docs.scipy.org/doc/numpy/reference/generated/numpy.tile.html
<br>
np.tile(A,reps)
<br>
reps: The number of repetitions of A along each axis. <br>
Note: If A.ndim > d, reps is promoted to A.ndim by pre-pending 1’s to it. Thus for an A of shape (2, 3, 4, 5), a reps of (2, 2) is treated as (1, 1, 2, 2).

In [0]:
b = np.array([[1, 2], [3, 4]])
# Repeat b in the second dimension twice (col-wise)
print(np.tile(b,2))
# The above line is equivalent to the following
#print(np.tile(b,(1,2)))

# Repeat b in the first dimension twice (row-wise)
print(np.tile(b,(2,1)))

Calculate anomaly with np.tile

In [0]:
#The following is one standard way with np.tile
print('Anomaly along the second dimension: ')
anom_matrix = arr - np.tile(row_mean,(3,1)).T # repeat for 3 times, axis = 1
print(anom_matrix)
#Alternatively
print('Alternatively, transpose before using np.tile')
print(np.shape(row_mean))
print(np.shape(row_mean.reshape((4,1))))
anom_matrix = arr - np.tile(row_mean.reshape((4,1)),3) # repeat for 3 times, axis = 1
print(anom_matrix)

### 3. Use list comprehension, lambda functions, or map()

Compare for-loop and list comprehension. 

In [0]:
# old way 
%%time
old_list = [0,1,2,3,4,5]
new_list = []
for i in old_list:
    if i > 2: #filter(i)
        new_list.append(i)
print(new_list)
        


In [0]:
# List comprehension
%%time
old_list = [0,1,2,3,4,5]
new_list=[]
new_list = [i for i in old_list if i > 2]
print(new_list)

**Quick conclusion**: 
One snapshot of time comsumption: <br> 
Old way (for-loop): 2.09 ms. <br>
Pythonic way (list comprehension): 1.72 ms.


**Lambda functions** adopt the form of:  


```
lambda arguments: expression
```



In [0]:
(lambda x: x + 1)(2)

In [0]:
# Old way of defining a function:
def g_old(x):
  return x**2

# Lambda function form:
g = lambda x: x**2
print(g(5))
print(g_old(5))

In [0]:
# Use lambda function with filter
a = np.arange(12)
final_list = list(filter(lambda x: (x%2 != 0) , a)) 
print(final_list) 

In [0]:
# Use lambda function with map
final_list = list(map(lambda x: (x*3+1) , a)) 
print(final_list) 

# Equivalent to:
#(lambda x: (x*3+1))(a)


## Masked Arrays

When an element of the mask is False, the corresponding element of the associated array is valid and is said to be unmasked. When an element of the mask is True, the corresponding element of the associated array is said to be masked (invalid).

In [0]:
a = np.arange(0,20,dtype=int)
#Mask out the elements larger than 10
a = np.ma.masked_where(a>10, a)

print(a)
print(a.data, a.mask)

In [0]:
print('Mean value of masked array a is: ')
print(a.mean())

Construct a mask by design: 

In [0]:
a = np.arange(0,20,dtype=int)
mask = [True for i in range(20)]
mask[0:3] = [False for i in range(3)]
a = np.ma.masked_where(mask,a)
a

In [0]:
# select the opposite 
a.mask == False

## Practice: Calculate means and anomalies with numpy 


Take a different dataset given under loaddir+'reanalysis_data/' or 'climate_data/'

> Calculate time mean and zonal mean of one surface field of your choice. <br>
> Subset the data you have (land/ocean; latitudinal bands; time), calculate the mean with masked array. <br> 
> Calculate the anomaly. 







In [0]:
# Read Dataset
#Please use netCDF4 (Dataset) if you want to use reanalysis 
f = Dataset(loaddir+'reanalysis_data/era5_global_201507.nc','r')


# Print all keywords
print(f)
lon = f.variables['longitude']
lat = f.variables['latitude']
time = f.variables['time']
t2m = f.variables['t2m']
land_sea_mask = f.variables['lsm'][0]
print(land_sea_mask)
print(lon[:10],lat[:10],time[:10])
# Check dimensions
np.shape(t2m)

# Calculate the time mean of 2m air temperature
t2m_time_mean = np.nanmean(t2m,axis=0)
print('Time Mean matrix:')
print(np.shape(t2m_time_mean))
t2m_zonal_mean = np.nanmean(t2m,axis=2)
print('Zonal Mean matrix:')
print(np.shape(t2m_zonal_mean))

# Land sea mask is a fractional number from 0 (sea) to 1 (land)
print(len(np.where((land_sea_mask > 0) & (land_sea_mask <1))[0]))


In [0]:
# You can take these codes and plot a time series
nts = 124 #Total time length
t2m_time_anom = t2m - np.repeat(t2m_time_mean[np.newaxis,:,:],nts,axis=0)
print(np.shape(t2m_time_anom))

ts = np.arange(nts)
# We are not adding anything to the plot yet. Let's save this for visualization part.
# We will come back to plotting on the forth day.
plt.plot(ts,t2m_time_anom[:,0,0])





Construct masked array of land/ocean at first time step (Taking only 60S to 60N). <br>
The data has fractional values in the range 0 (sea) to 1 (land). Here we can make a simple assumption to take grids with more than 50% land as a land grid, and vice versa.  

In [0]:
# Subset with land/ocean
t2m_land = np.ma.masked_array(t2m[0,12:61,:], mask=(land_sea_mask[12:61,:] < 0.5)) # mask out ocean
t2m_ocean = np.ma.masked_array(t2m[0,12:61,:], mask=(land_sea_mask[12:61,:] >= 0.5)) # mask out land

print('Mean temperature over land is ' + str(round(np.nanmean(t2m_land),2)))
print('Mean temperature over ocean is ' + str(round(np.nanmean(t2m_ocean),2)))

# Temperature anomaly over land, time = 0:
t2m_land_anom = t2m_land - np.nanmean(t2m_land)
print(np.shape(t2m_land_anom))



---



---



---



---



# Day 2: Pandas

Goals of this section: 

1.   Introducing pandas: Initializing pd.DataFrame. Build our examplary dataset with reanalysis.
2.   Manipulating DataFrame (selecting columns, combining, subsetting). 
3.   End-of-class practice: Build a long-form DataFrame from gridded data. <br>
Extra: Convert long-form back to 2D data and plot a map (with pd.pivot). 



Two good examples to follow to as introduction to pandas: <br>
*10-minutes to pandas*: <br> https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html <br>
*Python for Data Analysis, 2nd Edition* (ch05.ipynb): <br>
https://notebooks.azure.com/wesm/projects/python-for-data-analysis


## Preparation before start building DataFrame

### Read netcdf data from reanalysis datasets. 

Take a look at the Dataset we are going to wiork with today: 

In [4]:
filename1 = 'era5_global_201501.nc'
filename2 = 'era5_global_201507.nc'
data1 = Dataset(os.path.join(loaddir+'reanalysis_data/',filename1),'r')
data2 = Dataset(os.path.join(loaddir+'reanalysis_data/',filename2),'r')
data1
data1.variables['t2m']

<class 'netCDF4._netCDF4.Variable'>
int16 t2m(time, latitude, longitude)
    scale_factor: 0.0015416944188662582
    add_offset: 269.259101589314
    _FillValue: -32767
    missing_value: -32767
    units: K
    long_name: 2 metre temperature
unlimited dimensions: 
current shape = (124, 73, 144)
filling on

### Managing time objects in packages


**Datetime objects in netCDF4**:
*  dt
*  dt.year/month/day
*  dt.strftime

In [23]:
import netCDF4
time = data1.variables['time']
time[:]
### Do not declare the variable as datetime
dt = netCDF4.num2date(time[:],units=time.units,calendar=time.calendar)

# Print dt
print(list(map(lambda x:x.year,dt))[:5])

dt = netCDF4.num2date(time[0],units=time.units,calendar=time.calendar)
# Note: Remember to use .day not .date

# Print dt with strftime
print(dt)
print(dt.strftime('%m-%d-%Y %H:%M:%S'))
type(dt)
#print('The date is: ' + dt.strftime('%d'))


[2015, 2015, 2015, 2015, 2015]
2015-01-01 00:00:00
01-01-2015 00:00:00


datetime.datetime

**Additional**: datetime package and timedelta function. <br> 
https://docs.python.org/2/library/datetime.html

In [0]:
import datetime

dt = datetime.datetime(2015,1,1,0,0,0)
dt
dt.strftime('%Y/%m/%d %H:%M:%S')

**Timestamp features in pandas**:  pd.date_range <br>


*   Specify start and end date with form of MM/DD/YYYY or YYYYMMDD
*   Specify start date and period (and frequency: freq = 'D')



To learn more about frequency: 
http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [0]:
### Try different ways of initializing time stamps ###
dates = pd.date_range('20130101','20130108')
dates = pd.date_range('20130101', periods=10, freq='M') #Default frequency is 'D'
dates

In [0]:
# Convert date column in dataset to datetime object
guns = pd.read_csv('/content/drive/My Drive/cfr_bootcamp/Data/violence_data/Gun_Crimes_Heat_Map.csv')
guns['Date'] =  pd.to_datetime(guns['Date'])
guns['year'] = guns['Date'].dt.year
guns.head()

## Examplary dataset from sklearn (load_iris)
One example dataset from sklearn. We will go over basic pandas built-in functions based on that. 
*  .head/tail

In [0]:
data = skdata.load_iris()
df_iris = pd.DataFrame(data.data, columns = data.feature_names)
df_iris['target'] = data.target

#Rename the dataset
df_iris_new = df_iris.rename(columns={'sepal length (cm)':'sepal_len','sepal width (cm)':'sepal_wid','petal length (cm)':'petal_len','petal width (cm)':'petal_wid'})

# to view dataframes interactivly (Don't use print)
df_iris


* df.describe shows basic statistics of the given DataFrame. 

In [0]:
df_iris.describe()

*  Multiple lines output by your own design:

In [0]:
df_iris.agg(['std', 'mean','min','median','max'])

*  df.columns show column names
*  df.index show index range
*  df.idxmin (equivalent to np.argmin)

In [0]:
# blank cell
df_iris.idxmin()


Use value_count for a pd.Series (1D array from DataFrame will do): 


In [0]:
df_iris['target'].value_counts()

### Start by adding rows and columns.



* df.append() <br>

Using pandas.DataFrame.append(). <br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html <br>

pd.DataFrame append doesn't happen in-place. Be sure to use dfnew = df.append(dfnew). 

In [0]:
# Check the structure of df_iris
df_iris.tail()

In [0]:
iris_new = pd.DataFrame({'sepal length (cm)': [0], 'sepal width (cm)':[0],'petal length (cm)':[0],'petal width (cm)': [0],'target':[0]})
df_iris = df_iris.append(iris_new,ignore_index=True)
df_iris.tail()

Adding a column to pd.DataFrame 

*   Use square bracket
*   Use df.insert
*   Use df.assign



In [0]:
# You can add number, time, string as the new column
dates = pd.date_range('20130101', periods=len(df_iris), freq='5T') #Default frequency is 'D'
df_iris['time'] = dates


In [0]:
# With df.assign 
#df_iris.assign(time=dates)



In [0]:
# With df.insert
df_iris.insert(0,'time',dates)
df_iris

In [0]:
# The empty space below could be number, string, etc.
#df_iris['new_col'] = ['new' for li in range(len(df_iris))]
# Or add a new column with values given to specific rows 
df_iris['new_col'] = pd.Series([-1.2, -1.5, -1.7], index=[0,2,4])
df_iris.head()

### Let's remove the rows and columns we added

**Note:** Make sure you are removing a row/column already added. 

In [0]:
df_iris.drop([150]).tail()

In [0]:
df_iris.drop(columns=['new_col','time']).head()

### Arithmetic features

Add, subtract, multiply and divide are enables with pd.Series or pd.DataFrame of the mathcing dimensions.  

In [0]:
# Calculate the ratio between sepal length and width
sepal_rat = df_iris['sepal length (cm)'] / df_iris['sepal width (cm)']
sepal_rat.head()



### Subsetting based on some selection

*  Filter with square bracket
*  Filter with .loc/.iloc

In [0]:
# Select a subset with square bracket/loc
dfnew = df_iris[(df_iris.index >= 10)]
dfnew

In [0]:
# Subset with iloc
dfnew = df_iris.iloc[[0,3,4,5]]
dfnew

## Exercise: Manipulating a toy DataFrame

A toy DataFrame of population data (million) of two states in certain years are given. 

*   California, 1990, 29.8
*   California, 2000, 33.9
*   California, 2010, 37.3
*   Michigen, 1990, 9.3
*   Michigen, 2000, 9.9
*   Michigen, 2010, 9.9

Now start to explore a little bit: <br>
1.   Add one column of estimated population change from 2010 to 2019 in both states. 
*   California, 1990, 29.8, np.nan
*   California, 2000, 33.9, np.nan
*   California, 2010, 37.3, 2.6
*   Michigen, 1990, 9.3, np.nan
*   Michigen, 2000, 9.9, np.nan
*   Michigen, 2010, 9.9, 1.0
Note: Suggested to add elements with indexing. <br> 
2.   Construct this DataFrame with three rows for Illinois. Append to the above DataFrame. 
*   Illinois, 1990, 11.4, np.nan
*   Illinois, 2000, 12.4, np.nan
*   Illinois, 2010, 12.8, -0.1
3.   Calculate population of three states in year 2019 with arithmatic features. 

Reference: 
https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population <br> 
https://en.wikipedia.org/wiki/2000_United_States_Census



In [0]:
# Given the toy dataset here
data = {'state': ['California', 'California', 'California', 'Michigen', 'Michigen', 'Michigen'],
        'year': [1990, 2000, 2010, 1990, 2000, 2010],
        'pop': [29.8, 33.9, 37.3, 9.3, 9.9, 9.9]}
df_pop = pd.DataFrame(data)


In [0]:
# Start to explore 
df_pop['pop_diff'] = pd.Series([2.6,1.0],index=[2,5])
df_pop = df_pop[df_pop['year'] == 2010]
df_pop['pop'] + df_pop['pop_diff']

## Start building a first dataframe (with reanalysis)

One line example of DataFrame structure we want to use. 

In [0]:
df = pd.DataFrame({'year':[0],'month':[0],'day':[0],'hour':[0],'lon':[0],'lat':[0],'t2m':[0],'sp':[0],'d2m':[0],'lsm':[0]})
df

Converting this simple DataFrame to and from ndarray. 

In [0]:
# Converting DataFrame to numpy
new_dict = np.asarray(df)
type(new_dict)


In [0]:
# Converting numpy to DataFrame 
dfnew = pd.DataFrame.from_dict(new_dict)
type(dfnew)

Adding rows of pandas.DataFrame to the empty one: 

In [0]:
# For-loops: quite slow. 
%%time
filelist  = os.listdir(loaddir+'reanalysis_data/')
files     = fnmatch.filter(filelist, 'era5_global*.nc')

df = pd.DataFrame()
print('Total number of files are: '+str(len(files)))

for myfile in files: 
    filepath = '%s/%s'%(loaddir+'reanalysis_data/',myfile)
    f = Dataset(filepath,'r')
    # Alternatively: Extract year and mth from filename
    #ID = filepath.split('_global_', 1)[1]
    #my_year = ID[0:4]
    #my_mth = ID[4:6]
    lon = f['longitude']
    lat = f['latitude']
    time = f['time']
    # Just take 1 timestep in each month as an example
    nts = 1 #1 timestep
    nlon = len(lon)
    nlat = len(lat)
    print(nlon,nlat)
    #1st dimension: time; 2nd dimension: lat; 3rd dimension: lon.
    lsm = f['lsm'][0,:,:]
    for its in range(nts):
      dt = netCDF4.num2date(time[its],units=time.units,calendar=time.calendar)
      my_year = dt.strftime('%Y')
      my_mth = dt.strftime('%m')
      my_date = dt.strftime('%d')
      my_hour = dt.strftime('%#H')
      print('This is: '+ str(dt))
      
      t2m = f['t2m'][its,:,:]
      sp = f['sp'][its,:,:]
      d2m = f['d2m'][its,:,:]
      
      for ilon in np.arange(nlon):
        for ilat in np.arange(nlat):
            dfnew = pd.DataFrame({'year':[my_year],'month':[my_mth],'day':[my_date],'hour':[my_hour],'lon':[lon[ilon]],'lat':[lat[ilat]],'t2m':[t2m[ilat,ilon]],'sp':[sp[ilat,ilon]],'d2m':[d2m[ilat,ilon]],'lsm':[lsm[ilat,ilon]]})           
            df = df.append(dfnew, ignore_index=True)
            


In [0]:
df.head()

Here are two functions to compress the ndarray to 1D, in specified order. 
*   np.ravel
*   np.flatten

**Order**: C,F,A,K
- C Is Contiguous layout. Mathematically speaking, row major.
- F Is Fortran contiguous layout. Mathematically speaking, column major.
- A Is any order. Generally don't use this.
- K Is keep order. Generally don't use this.
https://stackoverflow.com/questions/27266338/what-does-the-order-parameter-in-numpy-array-do-aka-what-is-contiguous-order
<br>

**Difference?**  <br>
np.flatten returns a copy, but np.ravel does not (faster).
https://stackoverflow.com/questions/28930465/what-is-the-difference-between-flatten-and-ravel-functions-in-numpy



In [0]:
# Introducing np.ravel and np.flatten first:
arr = np.arange(12).reshape((4,3))
arr
arr.flatten(order='C')


array([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11])

Here's a workaround I and Jim came up with. It saves us from extremely time-consuming for-loops. 

In [0]:
# Let's get rid of the for-loop in spatial dimensions with np.ravel
%%time
filelist  = os.listdir(loaddir+'reanalysis_data/')
files     = fnmatch.filter(filelist, 'era5_global*.nc')

df = pd.DataFrame()
print('Total number of files are: '+str(len(files)))

for myfile in files: 
    filepath = '%s/%s'%(loaddir+'reanalysis_data/',myfile)
    f = Dataset(filepath,'r')

    lon = f['longitude']
    lat = f['latitude']
    mylon, mylat = np.meshgrid(lon,lat)
    mesh_lon = mylon.ravel()
    mesh_lat = mylat.ravel()
    time = f['time']
    # Take all time
    nts = len(time)
    # To compare with the above code
    #nts=1
    for its in range(nts):
      dt = netCDF4.num2date(time[its],units=time.units,calendar=time.calendar)
      my_year = dt.year
      my_mth = dt.month
      my_date = dt.day
      my_hour = dt.hour
      lsm = f['lsm'][its,:,:].ravel()
      t2m = f['t2m'][its,:,:].ravel()
      sp = f['sp'][its,:,:].ravel()
      d2m = f['d2m'][its,:,:].ravel()  

      dfnew = pd.DataFrame({'year':my_year,'month':my_mth,'day':my_date,'hour':my_hour,'lon':mesh_lon,'lat':mesh_lat,'t2m':t2m,'sp':sp,'d2m':d2m,'lsm':lsm})           
      df = df.append(dfnew, ignore_index=True)



In [0]:
df.iloc[300:350]


In [0]:
#Save DataFrame to csv (in homedir)
#df.to_csv(homedir+'era5_surface_longform_new.csv')

### Additional step: What if we also get rid of for-loop for time? 

In [0]:
# Let's get rid of the for-loop in time dimension also
%%time
filelist  = os.listdir(loaddir+'reanalysis_data/')
files     = fnmatch.filter(filelist, 'era5_global*.nc')

df = pd.DataFrame()
#print('Total number of files are: '+str(len(files)))

for myfile in files: 
    filepath = '%s/%s'%(loaddir+'reanalysis_data/',myfile)
    f = Dataset(filepath,'r')
    lon = f['longitude']
    lat = f['latitude']
    time = f['time']
    mylon, mylat, mytime = np.meshgrid(lon,lat,time)
    mesh_time = mytime.ravel()
    mesh_lon = mylon.ravel()
    mesh_lat = mylat.ravel()
    
    dt = netCDF4.num2date(mesh_time,units=time.units,calendar=time.calendar)
    my_year = [dtnew.year for dtnew in dt]
    my_mth =  [dtnew.month for dtnew in dt]
    my_date = [dtnew.day for dtnew in dt]
    my_hour = [dtnew.hour for dtnew in dt]
    
    lsm = f['lsm'][:,:,:].ravel()
    t2m = f['t2m'][:,:,:].ravel()
    sp = f['sp'][:,:,:].ravel()
    d2m = f['d2m'][:,:,:].ravel()
    

    dfnew = pd.DataFrame({'year':my_year,'month':my_mth,'day':my_date,'hour':my_hour,'lon':mesh_lon,'lat':mesh_lat,'t2m':t2m,'sp':sp,'d2m':d2m,'lsm':lsm})           
    df = df.append(dfnew, ignore_index=True)

**Conclusion:** It seems what is taking the longest time is the repeated data processing in large for-loops (1000 iterations) that piled up together.

## Manipulating Dataframes (Subsetting & Rearanging)

In [0]:
# Read from csv 
# Use first column as index_col, or one additional column of 'Unnamed:0' will appear. 
df = pd.read_csv(loaddir+'/era5_surface_longform.csv',index_col=0)
print(df.head())
print(len(df))


Sort along one dimension specified: 

In [0]:
dfnew = df.sort_values(by=['t2m'],ascending=True)
print('Ascending in order sorted by t2m')
print(dfnew.head())


Selecting columns: 


*   Use square bracket df['var']
*   Use df.var
*   Select multiple columns
*   Reindex the DataFrame


In [0]:
# blank cell
df.t2m.iloc[300:320]
#df[['year','month','date','hour','lon','lat','t2m']].head()


In [0]:
# Change the original order with reindex
df[0:20].reindex(list(range(0,20,2)))
#df[0:20].reindex([0,10,1,3,4,5])

### Rearanging DataFrame with pd.melt: <br> 
**pd.melt**: Rearange the dataset to select certain variables as identifier (id_vars) and others as values (value_vars). <br>


In [0]:
### Melt 
dfnew = pd.melt(df,id_vars = ['year','month','date','hour','lon','lat'],value_vars = ['t2m','d2m','sp'])
dfnew.head()


### Converting long-form data back to gridded data with pd.pivot. 
**df.pivot**: Convert long-form data back to 2D gridded data. See example later. 
**Note**: Don't worry about plotting part. Just take the code and plot a map (Not required content for this section). 

In [0]:
### To install basemap (does not come pre-loaded). Run following two lines only once.
#!apt-get -qq install libgeos-dev
#!pip install -qq https://github.com/matplotlib/basemap/archive/master.zip
from mpl_toolkits.basemap import Basemap

### This is not a good example of projection
import datetime
# Feel free to play around with the time below
mytime = datetime.datetime(2015,1,1,0,0,0)
df_time1 = df.loc[(df.month == 1) & (df.date == 1) & (df.hour == mytime.hour)]
df_pivot = df_time1.pivot(index='lat',columns='lon',values='t2m')
df_pivot.head()


In [0]:
### Now let's plot a map
# Also you may zoom in by changing boundaries 
m = Basemap(projection='cyl',resolution='c',llcrnrlat=-90,urcrnrlat=90,llcrnrlon=0,urcrnrlon=360)
m.drawcountries()
m.drawcoastlines()
t2m_time1 = df_pivot.to_numpy()
m.imshow(t2m_time1,cmap='Reds')
plt.title(mytime.strftime('%Y/%m/%d %H:%M:%S')+' T2M')


## Applying internal & external functions to DataFrame


### Row or column-wise function application (df.apply)

Basic built-in functions: 
https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html

**For internal functions:** Use attribute access. 
*  Mean, sum
*  Iterrows
*  Empty
*  Flexible comparison: gt, lt. 
*  Boolean reduction: All, any.

In [0]:
df['t2m'].mean()
df.head()

Loop over DataFrame with df.iterrows

In [0]:
for i, j in df.iloc[0:3].iterrows():
    print(i, j)

Test if the DataFrame is empty:

In [0]:
df.empty

Flexible comparison: Compare two DataFrame of the same shape with gt (greater than) or lt (less than). 

In [0]:
df['t2m'].lt(df['d2m'])

Boolean expression with query:

In [0]:
df.query('t2m < d2m')

Boolean reduction: Is there any True value (any)? Are they all True values (all)?

In [0]:
myloc = df['t2m'].lt(df['d2m'])
myloc.any()

# In case you are interested
# Meteorological concept: Supersaturation
#df.loc[myloc]


**For external functions:**
Use df.apply() to nake use of external functions. 

In [0]:
df[:10].apply(np.cumsum)

With lambda functions. 

In [0]:
# Lambda function with apply
df[['t2m','sp','d2m']].apply(lambda x: np.percentile(x,50))


## More Advanced Example: Subsetting

In [0]:
# Filter with conditions
df_NS60 = df[((df.lat >= -60) & (df.lat <= 60))]
#Equivalent to: 
df_NS60 = df.loc[((df.lat >= -60) & (df.lat <= 60))]
#Also equivalent to: Subsetting by finding the location index
df_loc_NS60 = df.iloc[np.where((df.lat >= -60) & (df.lat <= 60))]

df_loc_NS60.head()

In [0]:
### to select the inverse
df_polar = df[~df.index.isin(df_NS60.index)]
print(df_polar.head())
print(df_polar['lat'].head())

Lambda function: 
Subsetting to even number rows.

In [0]:
df.iloc[lambda x: x.index % 2 == 0].head()



---



---



---



---



## Practice: Building your own dataset (from gridded data to long-form data)

Start to build your own dataset with the following steps (30 minutes): 


*  Read data from a different gridded data: Either reanalysis_data, climate_data or crop_data/yield_simulations. <br>
*  Building pandas DataFrame from a structure that contains necessary information: Time (yr, mth, day, hour); Spatial coordinate (lat, lon); Variables. <br>
*  Explore the DataFrame by adding and removing rows/columns. Calcualte some statistics with built-in functions. Subset and calculate statistics again. Compare with the whole dataset. 
*  Save the DataFrame to your directory (homedir). 
*  Additional: Take the code for map plotting (Manipulating Dataframes Subsetting & Rearanging), and try to plot the variable of you chose 1) Snapshot at the first time step; 2) Temporal mean; 3) Temporal anomaly. 







In [0]:
# blank cell 
